<a href="https://colab.research.google.com/github/cibelerusso/MRASII/blob/main/C%C3%B3digos%20em%20Python/SME0823_Testes_de_hip%C3%B3teses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SME0823 Modelos de Regressão e Aprendizado Supervisionado II

# Testes de hipóteses na regressão logística


por **Cibele Russo**

**ICMC/USP - São Carlos SP**

In [5]:
import numpy as np
from numpy.random import default_rng
import statsmodels.api as sm
from scipy.special import expit  # sigmoid

def simulate_logistic_sample(n, beta0=0.0, beta1=0.0, x_generator=None, rng=None):
    """
    Gera uma amostra (y, X) de um modelo logístico:
        logit(p_i) = beta0 + beta1 * x_i
        y_i ~ Bernoulli(p_i)

    x_generator: função que recebe (n, rng) e retorna vetor x (n,)
    """
    rng = default_rng() if rng is None else rng
    if x_generator is None:
        # padrão: N(0,1)
        x = rng.normal(size=n)
    else:
        x = x_generator(n, rng)

    eta = beta0 + beta1 * x
    p = expit(eta)
    y = rng.binomial(1, p, size=n)

    # Matriz do modelo:
    X_full = np.column_stack((np.ones(n), x))  # [1, x]
    X_restricted = np.ones((n, 1))            # [1]
    return y, X_full, X_restricted, x


def fit_logit(y, X):
    """
    Ajusta modelo Logit com statsmodels (máxima verossimilhança).
    Retorna o objeto results (LogitResults).
    """
    model = sm.Logit(y, X)
    # método por padrão é Newton-Raphson; usar disp=0 pra silenciar
    res = model.fit(disp=0)
    return res

In [6]:
# ------------------------------------------------------------
# Estatísticas de teste
# ------------------------------------------------------------
def lr_test(y, X_full, X_restricted):
    """
    Teste da Razão de Verossimilhança (LR) para H0: restrição que remove a(s) col(s)
    de X_full que não estão em X_restricted (aqui: β1=0).
    Estatística: 2*(ll_full - ll_restricted) ~ qui2_{df}, df = p_full - p_restricted
    """
    res_full = fit_logit(y, X_full)
    res_rest = fit_logit(y, X_restricted)
    llf_full = res_full.llf
    llf_rest = res_rest.llf
    stat = 2.0 * (llf_full - llf_rest)
    df = X_full.shape[1] - X_restricted.shape[1]
    return stat, df


def wald_test(y, X_full, param_index=1):
    """
    Teste de Wald para H0: beta[param_index] = 0 no modelo completo.
    Estatística: (beta_hat / SE)^2 ~ qui2_1.
    Por padrão, param_index=1 testa a covariável x (assumindo [intercepto, x]).
    """
    res_full = fit_logit(y, X_full)
    beta_hat = res_full.params[param_index]
    se = np.sqrt(res_full.cov_params()[param_index, param_index])
    z = beta_hat / se
    stat = z**2
    df = 1
    return stat, df


def score_test_intercept_plus_x(y, X_restricted, x):
    """
    Teste de Score (LM) para H0: beta1 = 0 em modelo com intercepto + 1 covariável x.
    Implementação clássica do score sob H0:
        U = sum x_i * (y_i - mu0_i)
        I = sum x_i^2 * mu0_i*(1-mu0_i)   (informação para beta1 sob H0)
    Estatística: LM = U^2 / I ~ qui2_1.
    """
    # Ajusta modelo restrito: apenas intercepto
    res_rest = fit_logit(y, X_restricted)
    eta0 = (X_restricted @ res_rest.params)  # vetor n x 1 de preditores lineares sob H0
    mu0 = expit(eta0)

    resid = y - mu0
    U = np.sum(x * resid)
    I = np.sum((x**2) * mu0 * (1 - mu0))
    # Evitar divisão por zero:
    if I <= 1e-12:
        return np.nan, 1
    LM = (U**2) / I
    df = 1
    return LM, df



In [7]:

# ------------------------------------------------------------
# Simulação (tamanho e poder)
# ------------------------------------------------------------
def simulate_rejection_rates(
    n=200,
    beta0=0.0,
    beta1=0.0,
    reps=1000,
    alpha=0.05,
    x_generator=None,
    rng=None
):
    """
    Retorna as taxas de rejeição empíricas (LR, Wald, Score) ao nível alpha,
    para amostras geradas de um logit com parâmetros (beta0, beta1).
    """
    rng = default_rng() if rng is None else rng
    from scipy.stats import chi2

    lr_reject = 0
    wald_reject = 0
    score_reject = 0
    valid_score = 0

    chi2_crit_1 = chi2.ppf(1 - alpha, df=1)

    for _ in range(reps):
        y, X_full, X_rest, x = simulate_logistic_sample(
            n=n, beta0=beta0, beta1=beta1, x_generator=x_generator, rng=rng
        )

        # LR
        try:
            lr_stat, lr_df = lr_test(y, X_full, X_rest)
            if lr_stat >= chi2.ppf(1 - alpha, df=lr_df):
                lr_reject += 1
        except Exception:
            # pode falhar se separação/quase-separação ocorrer; ignoramos esta replicação
            pass

        # Wald
        try:
            wald_stat, _ = wald_test(y, X_full, param_index=1)
            if wald_stat >= chi2_crit_1:
                wald_reject += 1
        except Exception:
            pass

        # Score (LM) - versão intercepto + 1 x
        try:
            score_stat, _ = score_test_intercept_plus_x(y, X_rest, x)
            if np.isfinite(score_stat):
                valid_score += 1
                if score_stat >= chi2_crit_1:
                    score_reject += 1
        except Exception:
            pass

    results = {
        "alpha": alpha,
        "n": n,
        "reps": reps,
        "beta0": beta0,
        "beta1": beta1,
        "size_or_power": {
            "LR": lr_reject / reps,
            "Wald": wald_reject / reps,
            "Score": (score_reject / valid_score) if valid_score > 0 else np.nan,
            "Score_valid_fraction": valid_score / reps,
        },
    }
    return results

In [10]:
# ------------------------------------------------------------
# Exemplo de uso
# ------------------------------------------------------------
if __name__ == "__main__":
    rng = default_rng(2025)

    # Gerador de X: Normal(0,1)
    def xgen(n, rng):
        return rng.normal(size=n)

    # 1) Tamanho (nível empírico) sob H0: beta1 = 0
    res_size = simulate_rejection_rates(
        n=300, beta0=0.0, beta1=0.0, reps=2000, alpha=0.05, x_generator=xgen, rng=rng
    )
    print("=== Tamanho (H0: beta1=0) ===")
    print(res_size)

    # 2) Poder sob H1: beta1 != 0 (ex.: efeitos moderados)
    for b1 in [0.2, 0.4, 0.6]:
        res_power = simulate_rejection_rates(
            n=300, beta0=0.0, beta1=b1, reps=2000, alpha=0.05, x_generator=xgen, rng=rng
        )
        print(f"=== Poder (H1: beta1={b1}) ===")
        print(res_power)

=== Tamanho (H0: beta1=0) ===
{'alpha': 0.05, 'n': 300, 'reps': 2000, 'beta0': 0.0, 'beta1': 0.0, 'size_or_power': {'LR': 0.0495, 'Wald': 0.0475, 'Score': 0.048, 'Score_valid_fraction': 1.0}}
=== Poder (H1: beta1=0.2) ===
{'alpha': 0.05, 'n': 300, 'reps': 2000, 'beta0': 0.0, 'beta1': 0.2, 'size_or_power': {'LR': 0.409, 'Wald': 0.4005, 'Score': 0.405, 'Score_valid_fraction': 1.0}}
=== Poder (H1: beta1=0.4) ===
{'alpha': 0.05, 'n': 300, 'reps': 2000, 'beta0': 0.0, 'beta1': 0.4, 'size_or_power': {'LR': 0.922, 'Wald': 0.9205, 'Score': 0.9205, 'Score_valid_fraction': 1.0}}
=== Poder (H1: beta1=0.6) ===
{'alpha': 0.05, 'n': 300, 'reps': 2000, 'beta0': 0.0, 'beta1': 0.6, 'size_or_power': {'LR': 0.9995, 'Wald': 0.9995, 'Score': 0.9995, 'Score_valid_fraction': 1.0}}
